In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from model import Net
from dataset import DatasetLoader
import multiprocessing
from func_train_test import train, test
import matplotlib.pyplot as plt
import numpy as np

def show_sample_images(dataloader, classes, num_images=20):
    """
    Show sample images from the training dataset
    """
    # Get a batch of training images
    images, labels = next(iter(dataloader))
    
    # Create figure
    fig = plt.figure(figsize=(20, 10))
    
    # Show images
    for i in range(min(num_images, len(images))):
        ax = fig.add_subplot(4, 5, i + 1)
        
        # Convert tensor to numpy and transpose to correct format
        img = images[i].numpy().transpose(1, 2, 0)
        
        # Denormalize the image
        mean = np.array([0.5, 0.5, 0.5])
        std = np.array([0.5, 0.5, 0.5])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        ax.imshow(img)
        ax.set_title(f'Class: {classes[labels[i]]}')
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig('sample_training_images.png')
    plt.close()
    
    # Print class distribution in this batch
    unique_labels, counts = np.unique(labels.numpy(), return_counts=True)
    print("\nClass distribution in sample batch:")
    for label, count in zip(unique_labels, counts):
        print(f"{classes[label]}: {count} images")

def show_misclassified(images, predictions, targets, classes):
    """
    Show misclassified images in a grid
    """
    fig = plt.figure(figsize=(20, 10))
    for i in range(min(20, len(images))):  # Show up to 20 images
        ax = fig.add_subplot(4, 5, i + 1)
        # Convert tensor to numpy and transpose to correct format
        img = images[i].cpu().numpy().transpose(1, 2, 0)
        # Denormalize the image
        mean = np.array([0.5, 0.5, 0.5])
        std = np.array([0.5, 0.5, 0.5])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        ax.imshow(img)
        ax.set_title(f'Pred: {classes[predictions[i]]}\nTrue: {classes[targets[i]]}')
        ax.axis('off')
    plt.tight_layout()
    plt.savefig(f'misclassified_examples.png')
    plt.close()

def main():
    # Debug flag
    DEBUG = False  # Set to False to disable misclassified images visualization
    
    # Training settings
    EPOCHS = 50
    BATCH_SIZE = 128
    
    # CIFAR10 classes
    classes = ('plane', 'car', 'bird', 'cat', 'deer',
              'dog', 'frog', 'horse', 'ship', 'truck')
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Data loaders
    get_data = DatasetLoader(batch_size=BATCH_SIZE)
    train_loader = get_data.train_loader()
    test_loader = get_data.test_loader()
    
    # Show sample training images
    print("\nDisplaying sample training images...")
    show_sample_images(train_loader, classes)
    
    # Model
    model = Net().to(device)
    summary(model, input_size=(3, 32, 32))
    
    # Optimizer and criterion
    optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    # Training history
    train_losses = []
    test_losses = []
    train_accs = []
    test_accs = []
    
    for epoch in range(EPOCHS):
        print(f"\nEpoch: {epoch}")
        
        # Train
        train_loss, train_acc = train(model, device, train_loader, optimizer, criterion, epoch)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        
        # Test and collect misclassified examples
        test_loss, test_acc, misclassified = test(model, device, test_loader, criterion, debug=DEBUG)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
        # If in debug mode and we have misclassified examples, show them
        if DEBUG and misclassified and epoch % 5 == 0:  # Show every 5 epochs
            images, predictions, targets = misclassified
            show_misclassified(images, predictions, targets, classes)
        
        scheduler.step()
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Accuracy')
    plt.plot(test_accs, label='Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
 
multiprocessing.freeze_support()
main()

Using device: cuda


/home/xpz1/tsai_work/erav3/git_folder/ERAV3/S8/transforms.py:50: UserWarning: Argument 'fill_value' is not valid and will be ignored.
  A.CoarseDropout(max_holes=1, max_height=8, max_width=8, min_height=4, min_width=4, fill_value=0, mask_fill_value=None, p=0.5),
/home/xpz1/tsai_work/erav3/git_folder/ERAV3/S8/transforms.py:50: UserWarning: Argument 'mask_fill_value' is not valid and will be ignored.
  A.CoarseDropout(max_holes=1, max_height=8, max_width=8, min_height=4, min_width=4, fill_value=0, mask_fill_value=None, p=0.5),


100%|██████████| 170498071/170498071 [00:04<00:00, 36726588.58it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified

Displaying sample training images...

Class distribution in sample batch:
plane: 12 images
car: 15 images
bird: 11 images
cat: 12 images
deer: 11 images
dog: 19 images
frog: 15 images
horse: 10 images
ship: 14 images
truck: 9 images
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 64, 16, 16]           2,048
       BatchNorm2d-5           [-1, 64, 16, 16]             128
            Conv2d-6           [-1, 64, 32, 32]           2,048
       BatchNorm2d-7           [-1, 64, 32, 32]             128
              ReLU-8           [-1, 64, 32, 32]               0
            Conv2d-9           [-1, 

Epoch 0 Train: Loss=1.5663 Acc=42.25%: 100%|██████████| 391/391 [00:04<00:00, 90.19it/s]


...

Epoch 36 Train: Loss=0.3937 Acc=86.26%: 100%|██████████| 391/391 [00:04<00:00, 85.82it/s]



Test set: Average loss: 0.4508, Accuracy: 84.88%


Epoch: 37


Epoch 37 Train: Loss=0.3961 Acc=86.29%: 100%|██████████| 391/391 [00:04<00:00, 88.01it/s]



Test set: Average loss: 0.4526, Accuracy: 84.83%


Epoch: 38


Epoch 38 Train: Loss=0.3947 Acc=86.26%:  82%|████████▏ | 319/391 [00:03<00:00, 88.93it/s]